In [1]:
from Models.FLAN_T5 import Model

import numpy as np
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.pred_wrapper import PredictorWrapper


In [2]:
editor = Editor();

## Example formatting for FLAN_T5

In [3]:
flan = Model()

res = flan.get_results("potential energy is the energy stored in the car")

print(res)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'PE': 'Not Found', 'KE': 'Not Found', 'LCE': 'Not Found'}


## Valid Definitions

In [4]:
PE_ACCEPTABLE = [
    "energy at rest",
    "energy that a body has because of its position relative to other bodies",
    "energy stored in the car at the top of the initial drop",
]
PE_UNACCEPTABLE = [
    "energy in motion",
    "energy lost as the car goes down the hill",
    "opposite of kinetic energy",
]
PE_INSUFFICIENT = [
    "4 Joules at the top of the roller coaster",
    "measured in Joules",
    "equal to m*h*9.8",
]

KE_ACCEPTABLE = [
    "energy in motion",
    "energy that the car has because it is moving",
    "the work needed to accelerate the roller coaster car from rest",
]
KE_UNACCEPTABLE = [
    "energy at rest",
    "never lost nor gained as the car moves",
    "the opposite of potantial energy",
]
KE_INSUFFICIENT = [
    "4 Joules at the bottom of the roller coaster",
    "measured in Joules",
    "equal to .5*m*v^2",
]

LCE_ACCEPTABLE = [
    "energy cannot be created or destroyed, only transformed",
    "the total energy of an isolated system remains constant",
    "energy can be converted from one form to another, but never created or destroyed",
]
LCE_UNACCEPTABLE = [
    "energy can be created or destroyed, not transformed",
    "the total energy of an isolated system will change",
]
LCE_INSUFFICIENT = [
    "K1 + U1 = K2 + U2",
    "potential energy transforms into kinetic energy",
    "As the car goes down the hill, some energy is lost to friction as heat",
]


## Function for getting data and frequencies

In [5]:
flan = Model()

def getData(samples, actualLabel):
    data = {
        'PE':[],
        'KE':[],
        'LCE':[],
    }
    frequencies = {
        'PE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
        'KE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
        'LCE':{'Acceptable':0, 'Unacceptable':0, 'Insufficient':0},
    }
    fails = 0
    for sample in samples.data:
        d = flan.get_results(sample)
        label = d['PE'] if len(d['PE']) != 0 else d['KE'] if len(d['KE']) != 0 else d['LCE']
        if label != actualLabel:
            print("FAIL: " + sample)
            fails += 1
        data['PE'].append(d['PE'])
        data['KE'].append(d['KE'])
        data['LCE'].append(d['LCE'])
        frequencies['PE'][d['PE']] += 1
        frequencies['KE'][d['KE']] += 1
        frequencies['LCE'][d['LCE']] += 1
    print(str(fails) + " fails out of " + str(len(samples.data)) + " tests")
    return data, frequencies

# def runtest(text):
#     res = {}
#     d = Model(text)()
#     res['PE'] = d['PE']
#     res['KE'] = d['PE']
#     res['LCE'] = d['LCE']
#     return res


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Vocab + POS**

## Minimum Functionality Tests
### Check for correct definitions
Should return acceptable

In [7]:
samples = editor.template(
    "Potential energy is {pe_acceptable}", 
    pe_acceptable=PE_ACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE, nsamples=5, labels=0
)

data, frequencies = getData(samples, "Acceptable")

# data, freqs = getData(samples)
# test = MFT(samples.data, labels=samples.labels, name="Test vocab + POS", capability="Vocab + POS")
# test.run()
# test.summary()

FAIL: Potential energy is energy at rest


KeyError: 'Not Found'

### Check for incorrect definitions
should return unacceptable

In [30]:
samples = editor.template(
    "Potential energy is {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE, nsamples=5, labels=0
)

data, freqs = getData(samples, "Unacceptable")

FAIL: Potential energy is energy in motion
FAIL: Potential energy is opposite of kinetic energy
FAIL: Potential energy is energy in motion
FAIL: Potential energy is energy lost as the car goes down the hill
FAIL: Kinetic energy is the opposite of potantial energy
FAIL: Kinetic energy is energy at rest
FAIL: The Law of Conservation of energy states the total energy of an isolated system will change.
FAIL: The Law of Conservation of energy states the total energy of an isolated system will change.
FAIL: The Law of Conservation of energy states the total energy of an isolated system will change.
9fails out of 15tests


### Check insufficient definitions
should return insufficient

In [32]:
samples = editor.template(
    "Potential energy is {pe_insufficient}",
    pe_insufficient=PE_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_insufficient}",
    ke_insufficient=KE_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_insufficient}.",
    lce_insufficient=LCE_INSUFFICIENT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Insufficient")

FAIL: Potential energy is measured in Joules
FAIL: Potential energy is 4 Joules at the top of the roller coaster
FAIL: Potential energy is measured in Joules
FAIL: Potential energy is equal to m*h*9.8
FAIL: Kinetic energy is measured in Joules
FAIL: Kinetic energy is 4 Joules at the bottom of the roller coaster
FAIL: The Law of Conservation of energy states As the car goes down the hill, some energy is lost to friction as heat.
FAIL: The Law of Conservation of energy states potential energy transforms into kinetic energy.
FAIL: The Law of Conservation of energy states potential energy transforms into kinetic energy.
FAIL: The Law of Conservation of energy states As the car goes down the hill, some energy is lost to friction as heat.
10fails out of 15tests


## Directional Tests

### Correct definitions followed by incorrect definition
correctness should not go up

In [33]:
samples = editor.template(
    "Potential energy is {pe_acceptable}. potential energy is {pe_unacceptable}. Kinetic energy is {ke_acceptable}. kinetic energy is {ke_unacceptable}. The Law of Conservation of energy states {lce_acceptable}."
    "The Law of Conservation of energy states {lce_unacceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    nsamples=50, labels=0
)

data, freqs = getData(samples)
print (samples.data)

TypeError: getData() missing 1 required positional argument: 'actualLabel'

### Incorrect Definition followed by correct definition
correctness should not go down

In [34]:
# code
samples = editor.template(
    "Potential energy is {pe_unacceptable}. Potential energy is {pe_acceptable}. Kinetic energy is {ke_unacceptable}. Kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_unacceptable}."
    "The Law of Conservation of energy states {lce_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    nsamples=50, labels=0
)


data, freqs = getData(samples)

TypeError: getData() missing 1 required positional argument: 'actualLabel'

### Insufficient definition followed by correct definition
Correctness should not go down

In [35]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_acceptable}. Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_acceptable}. The Law of Conservation of energy states {lce_insufficient}."
    "The Law of Conservation of energy states {lce_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, ke_acceptable=KE_ACCEPTABLE, lce_acceptable=LCE_ACCEPTABLE,
    pe_insufficient=PE_INSUFFICIENT, ke_insufficient=KE_INSUFFICIENT, lce_insufficient=LCE_INSUFFICIENT,
    nsamples=50, labels=0
)

data, freqs = getData(samples, "Acceptable")

TypeError: getData() missing 1 required positional argument: 'actualLabel'

### Insufficient definition followed by incorrect definition
Should result in Acceptable

In [ ]:
samples = editor.template(
    "Potential energy is {pe_insufficient}. Potential energy is {pe_unacceptable}. Kinetic energy is {ke_insufficient}. Kinetic energy is {ke_unacceptable}. The Law of Conservation of energy states {lce_insufficient}."
    "The Law of Conservation of energy states {lce_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE, ke_unacceptable=KE_UNACCEPTABLE, lce_unacceptable=LCE_UNACCEPTABLE,
    pe_insufficient=PE_INSUFFICIENT, ke_insufficient=KE_INSUFFICIENT, lce_insufficient=LCE_INSUFFICIENT,
    nsamples=50, labels=0
)

data, freqs = getData(samples, "Acceptable")


# Negation

## MFTs

### Negated correct definition
Should return unacceptable

In [36]:
samples = editor.template(
    "Potential energy is not {pe_acceptable}",
    pe_acceptable=PE_ACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is not {ke_acceptable}",
    ke_acceptable=KE_ACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy does not state {lce_acceptable}.",
    lce_acceptable=LCE_ACCEPTABLE, nsamples=5, labels=0
)

data, frequencies = getData(samples, "Unacceptable")


FAIL: Potential energy is not energy that a body has because of its position relative to other bodies
FAIL: Potential energy is not energy at rest
FAIL: Potential energy is not energy stored in the car at the top of the initial drop
FAIL: Potential energy is not energy at rest
FAIL: Kinetic energy is not energy that the car has because it is moving
FAIL: Kinetic energy is not the work needed to accelerate the roller coaster car from rest
FAIL: Kinetic energy is not the work needed to accelerate the roller coaster car from rest
FAIL: Kinetic energy is not the work needed to accelerate the roller coaster car from rest
FAIL: The Law of Conservation of energy does not state the total energy of an isolated system remains constant.
FAIL: The Law of Conservation of energy does not state the total energy of an isolated system remains constant.
FAIL: The Law of Conservation of energy does not state the total energy of an isolated system remains constant.
11fails out of 15tests


### Negation of incorrect definition
Should return insufficient

In [ ]:
samples = editor.template(
    "Potential energy is not {pe_unacceptable}",
    pe_unacceptable=PE_UNACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is not {ke_unacceptable}",
    ke_unacceptable=KE_UNACCEPTABLE, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy does not state {lce_unacceptable}.",
    lce_unacceptable=LCE_UNACCEPTABLE, nsamples=5, labels=0
)

data, frequencies = getData(samples, "Insufficient")

### Negation of insufficient
Should return insufficient

In [ ]:
samples = editor.template(
    "Potential energy is not {pe_insufficient}",
    pe_insufficient=PE_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is not {ke_insufficient}",
    ke_insufficient=KE_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy does not state {lce_insufficient}.",
    lce_insufficient=LCE_INSUFFICIENT, nsamples=5, labels=0
)

data, frequencies = getData(samples, "Insufficient")


# Robustness

## INV

### Misspelling of words
Should return the same value

#### Correct definitions

In [ ]:
PE_TYPO_CORRECT = [
    "energy that body has becuse of its posiion relative to other bdies."
    "energy store in te car at the tp of the initl drp."
]
KE_TYPO_CORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng."
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest."
]
LCE_TYPO_CORRECT = [
    "that energy canot be create or destroy, ony transfomed."
    "the totl energy of an isolated systm remain constnt."
]

samples = editor.template(
    "Potential energy is {pe_typo_correct}",
    pe_typo_correct=PE_TYPO_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_typo_correct}",
    ke_typo_correct=KE_TYPO_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_typo_correct}.",
    lce_typo_correct=LCE_TYPO_CORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Insufficient")

#### Incorrect definitions

In [37]:
PE_TYPO_INCORRECT = [
    "Potential energy i energy in moton.",
    "Potentia energy is the energy lose as th car go down the hil."
]
KE_TYPO_INCORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng.",
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.",

]
LCE_TYPO_INCORRECT = [
    "LCE says that energy canot be create or destroy, ony transfomed.",
    "LCE state the totl energy of an isolated systm remain constnt.",
]

samples = editor.template(
    "Potential energy is {pe_typo_incorrect}",
    pe_typo_incorrect=PE_TYPO_INCORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_typo_incorrect}",
    ke_typo_incorrect=KE_TYPO_INCORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_typo_incorrect}.",
    lce_typo_incorrect=LCE_TYPO_INCORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Insufficient")


FAIL: Potential energy is energy that body has becuse of its posiion relative to other bdies.energy store in te car at the tp of the initl drp.
FAIL: Potential energy is energy that body has becuse of its posiion relative to other bdies.energy store in te car at the tp of the initl drp.
FAIL: Potential energy is energy that body has becuse of its posiion relative to other bdies.energy store in te car at the tp of the initl drp.
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest.
FAIL: Kinetic energy is Kinetic enery is energy tht the car hs bcase i is movng.Kinetic ergy is the wrk ne

#### Insufficient definitions

In [38]:
PE_TYPO_INSUFFICIENT = [
    "The potentia energ at the top of te rollercoaer is 4.9 juls.",
    "Potential enrgy is measur in jules.",
]
KE_TYPO_INSUFFICIENT = [
    "The kinetic energy at the botom of the hile is 4.8 joul.",
    "Kinetic nergy transfourms into heet thrugh fricton",
]
LCE_TYPO_INSUFFICIENT = [
    "The potensial energy trainsforms in to kinetic energy becuz of the law of conservashun of energy.",
    "As the car goes down the hill, some energ is lost to fricshiun as heat.",
]

samples = editor.template(
    "Potential energy is {pe_typo_insufficient}",
    pe_typo_insufficient=PE_TYPO_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_typo_insufficient}",
    ke_typo_insufficient=KE_TYPO_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_typo_insufficient}.",
    lce_typo_insufficient=LCE_TYPO_INSUFFICIENT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Insufficient")


FAIL: Potential energy is The potentia energ at the top of te rollercoaer is 4.9 juls.
FAIL: Potential energy is The potentia energ at the top of te rollercoaer is 4.9 juls.
FAIL: Potential energy is Potential enrgy is measur in jules.
FAIL: Potential energy is Potential enrgy is measur in jules.
FAIL: Kinetic energy is The kinetic energy at the botom of the hile is 4.8 joul.
FAIL: Kinetic energy is Kinetic nergy transfourms into heet thrugh fricton
FAIL: Kinetic energy is The kinetic energy at the botom of the hile is 4.8 joul.
FAIL: Kinetic energy is Kinetic nergy transfourms into heet thrugh fricton
FAIL: The Law of Conservation of energy states As the car goes down the hill, some energ is lost to fricshiun as heat..
FAIL: The Law of Conservation of energy states As the car goes down the hill, some energ is lost to fricshiun as heat..
10fails out of 15tests


# Taxonomy

## INV
### Replacing words with synonyms
#### Correct definition

In [ ]:
PE_SYNONYM_CORRECT = [
    "energy that body has becuse of its posiion relative to other bdies."
    "energy store in te car at the tp of the initl drp."
]
KE_SYNONYM_CORRECT = [
    "Kinetic enery is energy tht the car hs bcase i is movng."
    "Kinetic ergy is the wrk neded to acelerate the rolercoaser car from rest."
]
LCE_SYNONYM_CORRECT = [
    "that energy canot be create or destroy, ony transfomed."
    "the totl energy of an isolated systm remain constnt."
]

samples = editor.template(
    "Potential energy is {pe_synonym_correct}",
    pe_synonym_correct=PE_SYNONYM_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_synonym_correct}",
    ke_synonym_correct=KE_SYNONYM_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_synonym_correct}.",
    lce_synonym_correct=LCE_SYNONYM_CORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Acceptable")

#### Incorrect definition

In [ ]:
PE_SYNONYM_INCORRECT = [
    "Potential energy is the energy lost as the car rolls down the hill.",
    "Potential energy is the opposite of kinetic energy.",
]
KE_SYNONYM_INCORRECT = [
    "Kinetic energy is never lost or gained as the car moves through the rollercoaster.",
    "Kinetic energy is the opposite of potential energy.",
]
LCE_SYNONYM_INCORRECT = [
    "LCE says that energy can be made and taken.",
    "LCE states that the energy of a closed system will differ.",
    "LCE says that in an open system, energy is constant",
]

samples = editor.template(
    "Potential energy is {pe_synonym_incorrect}",
    pe_synonym_incorrect=PE_SYNONYM_INCORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_synonym_incorrect}",
    ke_synonym_incorrect=KE_SYNONYM_INCORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_synonym_incorrect}.",
    lce_synonym_incorrect=LCE_SYNONYM_INCORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Unacceptable")

#### Insufficient definition

In [39]:
PE_SYNONYM_INSUFFICIENT = [
    "The potential energy at the tip of the rollercoaster is 4.9 joules.",
    "Potential energy is calculated in joules.",
]
KE_SYNONYM_INSUFFICIENT = [
    "There is higher kinetic energy at the bottom of the hill than at the top.",
    "Kinetic energy turns into heat through friction",
]
LCE_SYNONYM_INSUFFICIENT = [
    "As the car goes down the hill, some energy does away to friction as heat.",
    "If there were no friction, the energy would stay the same.",
]

samples = editor.template(
    "Potential energy is {pe_synonym_insufficient}",
    pe_synonym_insufficient=PE_SYNONYM_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_synonym_insufficient}",
    ke_synonym_insufficient=KE_SYNONYM_INSUFFICIENT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_synonym_insufficient}.",
    lce_synonym_insufficient=LCE_SYNONYM_INSUFFICIENT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Insufficient")

FAIL: Potential energy is Potential energy is calculated in joules.
FAIL: Potential energy is The potential energy at the tip of the rollercoaster is 4.9 joules.
FAIL: Potential energy is Potential energy is calculated in joules.
FAIL: Kinetic energy is There is higher kinetic energy at the bottom of the hill than at the top.
FAIL: Kinetic energy is There is higher kinetic energy at the bottom of the hill than at the top.
FAIL: Kinetic energy is There is higher kinetic energy at the bottom of the hill than at the top.
FAIL: Kinetic energy is Kinetic energy turns into heat through friction
FAIL: Kinetic energy is Kinetic energy turns into heat through friction
FAIL: The Law of Conservation of energy states If there were no friction, the energy would stay the same..
FAIL: The Law of Conservation of energy states As the car goes down the hill, some energy does away to friction as heat..
10fails out of 15tests


# Fairness

## INV
## Test for words being replaced in Spanish
#### Correct definitions

In [40]:
PE_SPANISH_CORRECT = [
    "Potential energy is energy that a body has because of its position relativa a other bodies.",
    "Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.",
    "Potential energy is energy that has the potential to convertirse en otro form of energy.",
]
KE_SPANISH_CORRECT = [
    "Kinetic energy is the work necesario para accelerate the rollercoaster car from rest.",
    "Kinetic energy is determined by the mass del carro and the velocity with which it is moving.",
    "Kinetic energy quantifies the work an object performs debido a su movimiento",
]
LCE_SPANISH_CORRECT = [
    "LCE states that energy can be convertida from one form to another, but never created or destroyed.",
    "LCE says that if there were no friction, the potential energy en la cima del rollercoaster would be the same as the kinetic energy at the bottom of the drop.",
    "LCE is a physical law that states that energy no se puede crear or destroyed but only transformed",
]

samples = editor.template(
    "Potential energy is {pe_spanish_correct}",
    pe_spanish_correct=PE_SPANISH_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "Kinetic energy is {ke_spanish_correct}",
    ke_spanish_correct=KE_SPANISH_CORRECT, nsamples=5, labels=0
)
samples += editor.template(
    "The Law of Conservation of energy states {lce_spanish_correct}.",
    lce_spanish_correct=LCE_SPANISH_CORRECT, nsamples=5, labels=0
)

data, freqs = getData(samples, "Acceptable")


FAIL: Potential energy is Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.
FAIL: Potential energy is Potential energy is the stored energy of the rollercoaster car when no se esta moviendo.
FAIL: Potential energy is Potential energy is energy that has the potential to convertirse en otro form of energy.
FAIL: Potential energy is Potential energy is energy that has the potential to convertirse en otro form of energy.
FAIL: Potential energy is Potential energy is energy that a body has because of its position relativa a other bodies.
FAIL: Kinetic energy is Kinetic energy quantifies the work an object performs debido a su movimiento
FAIL: Kinetic energy is Kinetic energy is determined by the mass del carro and the velocity with which it is moving.
FAIL: Kinetic energy is Kinetic energy quantifies the work an object performs debido a su movimiento
FAIL: The Law of Conservation of energy states LCE is a physical law that states that energy no se pued